In [25]:
! pip install -qU torch transformers datasets faiss-cpu pandas openpyxl

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.7.0+cu118 requires torch==2.7.0+cu118, but you have torch 2.9.1 which is incompatible.
torchvision 0.22.0 requires torch==2.7.0, but you have torch 2.9.1 which is incompatible.


In [37]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from datasets import Dataset

## 1. Wczytanie danych

In [9]:
xlsx_path = "tb_chunks.xlsx"

df = pd.read_excel(xlsx_path) 
print(df.shape)
print(df.columns)

(2836, 21)
Index(['doc_id', 'file_name', 'chunk_id', 'chunk_kind', 'kind', 'section_path',
       'annex_id', 'chapter_id', 'article_id', 'text', 'page_start',
       'page_end', 'reg_no', 'supplement', 'effective_from', 'jurisdiction',
       'tokens', 'level', 'parent_path', 'anchor', 'part_index'],
      dtype='object')


In [11]:
df.head()

,doc_id,file_name,chunk_id,chunk_kind,kind,section_path,annex_id,chapter_id,article_id,text,...,page_end,reg_no,supplement,effective_from,jurisdiction,tokens,level,parent_path,anchor,part_index
0,ECE 160R02-00 EDR cons,ECE 160R02-00 EDR cons.pdf,leaf:ECE 160R02-00 EDR cons:2.49.,LEAF,SECTION,2.49.,NaN,NaN,NaN,"2.49. ""Stability control"" means any device tha...",...,5,UN R160,02 series of amendments,NaT,UNECE,27,2.0,2.,UN R160 §2.49. (eff. n/a),NaN
1,ECE 160R02-00 EDR cons,ECE 160R02-00 EDR cons.pdf,leaf:ECE 160R02-00 EDR cons:3.2.3.,LEAF,SECTION,3.2.3.,NaN,NaN,NaN,3.2.3. Instructions for retrieving data from t...,...,6,UN R160,02 series of amendments,NaT,UNECE,13,3.0,3.2.,UN R160 §3.2.3. (eff. n/a),NaN
2,ECE 161R00-04 Unauthorized use and device agai...,ECE 161R00-04 Unauthorized use and device agai...,leaf:ECE 161R00-04 Unauthorized use and device...,LEAF,SECTION,4.5.,NaN,NaN,NaN,"4.5. \nIf a type conforms to a type approved, ...",...,5,UN R161,Supplement 4 to the original version of Regula...,2024-06-15,UNECE,117,2.0,4.,UN R161 §4.5. (eff. 2024-06-15),NaN
3,ECE 162R00-05 Immobilizers cons,ECE 162R00-05 Immobilizers cons.pdf,leaf:ECE 162R00-05 Immobilizers cons:5.1.9.,LEAF,SECTION,5.1.9.,NaN,NaN,NaN,5.1.9. \nIt shall not be possible for an immob...,...,7,UN R162,Supplement 5 to the original version of the Re...,2024-06-15,UNECE,157,3.0,5.1.,UN R162 §5.1.9. (eff. 2024-06-15),NaN
4,ECE 160R02-00 EDR cons,ECE 160R02-00 EDR cons.pdf,leaf:ECE 160R02-00 EDR cons:2.56.,LEAF,SECTION,2.56.,NaN,NaN,NaN,"2.56. ""Time to deploy, side air bag/curtain"" m...",...,5,UN R160,02 series of amendments,NaT,UNECE,50,2.0,2.,UN R160 §2.56. (eff. n/a),NaN


In [12]:
TEXT_COL = "text"

In [13]:
# proste czyszczenie: wyrzuć wiersze bez tekstu
df = df[df[TEXT_COL].notna()].reset_index(drop=True)
print("Po czyszczeniu:", df.shape)

Po czyszczeniu: (2836, 21)


## 2. Model embeddingowy

In [1]:
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

d:\users\sf34875\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\users\sf34875\.cache\huggingface\hub\models--sentence-transformers--multi-qa-mpnet-base-dot-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [5]:
def cls_pooling(model_output):
    # [batch, seq_len, hidden] → [batch, hidden], bierzemy token [CLS]
    return model_output.last_hidden_state[:, 0]

In [6]:
@torch.no_grad()
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,  # na wszelki wypadek
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    embeddings = cls_pooling(model_output)
    return embeddings.cpu().numpy()

In [14]:
test_emb = get_embeddings([df[TEXT_COL].iloc[0]])
print(test_emb.shape)  # (1, 768)

(1, 768)


In [15]:
test_emb

array([[-4.72052917e-02, -6.56372547e-01, -2.76724070e-01,
        -3.46933156e-01,  1.47241503e-01, -6.83597624e-02,
         2.71482021e-01,  2.72080842e-02,  9.88794118e-02,
         1.87647566e-01, -2.17194501e-02, -6.02854276e-03,
        -1.71285830e-02,  8.16682056e-02,  1.22139983e-01,
        -2.02150926e-01,  2.59146661e-01,  1.14800714e-01,
         1.64700344e-01,  1.29657209e-01, -1.98401928e-01,
        -1.04573220e-01, -2.84783840e-01, -4.42998521e-02,
        -2.24791855e-01,  1.39552444e-01,  4.33116794e-01,
        -1.29539534e-01,  4.94761532e-03, -2.81166375e-01,
         2.92516470e-01,  3.40127915e-01,  1.46494836e-01,
         1.25227123e-01, -9.76357842e-05, -4.22992222e-02,
         9.17381942e-02, -1.92948639e-01, -1.01875234e-02,
         3.40387970e-01, -8.20228457e-02, -3.25415805e-02,
        -1.65558070e-01,  5.67736290e-02, -4.21523675e-02,
         3.02376021e-02, -1.10933818e-01,  1.67308107e-01,
         5.87347038e-02,  1.42638311e-01,  3.10694695e-0

## 3. Zamiana na HuggingFace Dataset + policzenie embeddings

In [16]:
# pandas -> HF Dataset
ds = Dataset.from_pandas(df, preserve_index=False)

In [19]:
def embed_batch(batch):
    embs = get_embeddings(batch[TEXT_COL])          # (batch, hidden)
    embs = embs.astype("float32")                   # <-- DODAJ TO
    return {"embeddings": embs}

embeddings_ds = ds.map(
    embed_batch,
    batched=True,
    batch_size=32,
)

Map:   0%|          | 0/2836 [00:00<?, ? examples/s]

In [20]:
embeddings_ds.save_to_disk("regs_chunks_with_embeddings")

Saving the dataset (0/1 shards):   0%|          | 0/2836 [00:00<?, ? examples/s]

Do wczytania embeddingów i indeksu nie od zera

In [ ]:
#embeddings_ds = Dataset.load_from_disk("regs_chunks_with_embeddings")
#embeddings_ds.load_faiss_index("embeddings", "faiss_index.bin")

## 4. FAISS index

Ponieważ zbiór danych zawiera około 2800 embeddingów (znacznie poniżej progu 100k), zgodnie z zaleceniami FAISS (‘Guidelines to choose an index’) najlepszym rozwiązaniem jest indeks typu IndexFlatIP, który zapewnia dokładne wyszukiwanie bez konieczności uczenia struktury i bez strat jakości. Zastosowanie go dało optymalny kompromis między prostotą implementacji, szybkością oraz jakością wyników.

In [21]:
embeddings_ds.add_faiss_index(column="embeddings")
embeddings_ds.save_faiss_index("embeddings", "faiss_index.bin")

  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
print(type(embeddings_ds[0]["embeddings"]))
print(np.array(embeddings_ds[0]["embeddings"]).shape)

<class 'list'>
(768,)


In [26]:
print("Liczba wierszy:", embeddings_ds.num_rows)

Liczba wierszy: 2836


In [28]:
import os

def get_dir_size(path="."):
    total = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            fp = os.path.join(root, f)
            total += os.path.getsize(fp)
    return total

path = "regs_chunks_with_embeddings"
size_bytes = get_dir_size(path)
size_mb = size_bytes / (1024 * 1024)

print(f"Rozmiar datasetu HF: {size_mb:.2f} MB")


Rozmiar datasetu HF: 11.31 MB


In [29]:
import os

size_bytes = os.path.getsize("faiss_index.bin")
print(f"Rozmiar FAISS indexu: {size_bytes / (1024*1024):.2f} MB")


Rozmiar FAISS indexu: 8.31 MB


## 5. Funkcja semantic_search(query, k)

In [23]:
def embed_query(query: str) -> np.ndarray:
    q_emb = get_embeddings([query])  # shape (1, 768)
    return q_emb.astype("float32")

def semantic_search(query: str, k: int = 5):
    q_emb = embed_query(query)
    scores, samples = embeddings_ds.get_nearest_examples(
        "embeddings",
        q_emb,
        k=k,
    )
    # zamiana samples -> dataframe, żeby było wygodnie
    res_df = pd.DataFrame.from_dict(samples)
    res_df["score"] = scores
    res_df.sort_values("score", ascending=False, inplace=True)
    return res_df

## ZADANIE 2 - 2a i 2b („kilka wyszukiwań + analiza błędów”)

In [30]:
def pretty_search(query: str, k: int = 5):
    print(f"\n=== QUERY: {query} ===\n")
    results = semantic_search(query, k=k)
    for i, (_, row) in enumerate(results.iterrows(), start=1):
        print(f"#{i}  SCORE: {row['score']:.3f}")
        print(f"ANCHOR: {row.get('anchor')}")
        print(f"KIND: {row.get('chunk_kind')}, REG: {row.get('reg_no')}, SECTION: {row.get('section_path')}")
        print(f"TEXT:\n{row[TEXT_COL][:600]}...\n")
        print("-" * 80)

In [31]:
pretty_search("requirements for trailer direction indicator failure tell-tale", k=5)


=== QUERY: requirements for trailer direction indicator failure tell-tale ===

#1  SCORE: 29.622
ANCHOR: UN R048 §6.5.1. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 6.5.1.
TEXT:
6.5.1. 
Presence (see figure below) 
 
Mandatory. Types of direction-indicator lamps fall into categories (1, 1a, 1b, 
2a, 2b, 5 and 6) the assembly of which on one vehicle constitutes an 
arrangement ("A" and "B").  
 
Arrangement "A" shall apply to all motor vehicles.  
 
Arrangement "B" shall apply to trailers only....

--------------------------------------------------------------------------------
#2  SCORE: 29.493
ANCHOR: UN R048 §6.5.8. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 6.5.8.
TEXT:
6.5.8. 
Tell-tale 
Operating 
tell-tale 
mandatory 
for 
direction-indicator 
lamps 
of 
categories 1, 1a, 1b, 2a and 2b. It may be visual or auditory or both. If it is 
visual it shall be a flashing light which, at least in the event of the 
malfunction of any of these direction-indicator lamps, 

In [32]:
pretty_search("what is the minimum deceleration for service braking", k=5)


=== QUERY: what is the minimum deceleration for service braking ===

#1  SCORE: 30.073
ANCHOR: UN R13-H §2.9.3. (eff. 2023-04-01)
KIND: LEAF, REG: UN R13-H, SECTION: 2.9.3.
TEXT:
2.9.3. the braking force can be easily regulated with sufficient precision....

--------------------------------------------------------------------------------
#2  SCORE: 29.904
ANCHOR: UN R13-H Annex 3 - part 6 (eff. 2023-04-01)
KIND: ANNEX_PART, REG: UN R13-H, SECTION: None
TEXT:
case where the failure condition causes the electric component to deliver its maximum braking force. 
2.3. Parking braking system
2.3.1. The parking braking system shall be capable of holding the laden vehicle stationary on a 20 per cent up or down
gradient.
2.3.2. On vehicles to which the coupling of a trailer is authorized, the parking braking system of the motor vehicle shall
be capable of holding the combination of vehicles stationary on a 12 per cent up or down gradient.
2.3.3. If the control device is manual, the force appli

In [33]:
pretty_search("cybersecurity management system requirements", k=5)


=== QUERY: cybersecurity management system requirements ===

#1  SCORE: 35.411
ANCHOR: UN R155 §7.2.2.5. (eff. n/a)
KIND: LEAF, REG: UN R155, SECTION: 7.2.2.5.
TEXT:
7.2.2.5. The vehicle manufacturer shall be required to demonstrate how their Cyber Security Management
System will manage dependencies that may exist with contracted suppliers, service providers or manufacturer’s
sub-organizations in regards of the requirements of paragraph 7.2.2.2....

--------------------------------------------------------------------------------
#2  SCORE: 35.354
ANCHOR: UN R155 Annex 5 - part 8 (eff. n/a)
KIND: ANNEX_PART, REG: UN R155, SECTION: None
TEXT:
rue if software has not been
tested to verify that known bad code/bugs is
not present and reduce the risk of unknown
bad code/bugs being present
M23
Cybersecurity best practices for software and
hardware development shall be followed.
Cybersecurity testing with adequate coverage
28.2
Using remainders from development (e.g.
debug ports, JTAG ports, 

In [34]:
pretty_search("regulation about automatic headlamp levelling", k=5)


=== QUERY: regulation about automatic headlamp levelling ===

#1  SCORE: 28.562
ANCHOR: UN R121 §40. (eff. 2023-09-24)
KIND: LEAF, REG: UN R121, SECTION: 40.
TEXT:
40.
Headlamp levelling
13/
Control
No...

--------------------------------------------------------------------------------
#2  SCORE: 27.691
ANCHOR: UN R121 §40. (eff. 2023-09-24)
KIND: PACK, REG: UN R121, SECTION: 40.
TEXT:
40.
40.
Headlamp levelling
13/
Control
No...

--------------------------------------------------------------------------------
#3  SCORE: 22.418
ANCHOR: UN R048 §2.2.1.3. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 2.2.1.3.
TEXT:
2.2.1.3. 
The headlamp-levelling system;...

--------------------------------------------------------------------------------
#4  SCORE: 17.510
ANCHOR: UN R048 §6.2.6.2.1. (eff. 2024-09-22)
KIND: LEAF, REG: UN R048, SECTION: 6.2.6.2.1.
TEXT:
6.2.6.2.1. 
In the case where a headlamp levelling device is necessary to satisfy the 
requirements of paragraphs 6.2.6.1.1. and 